In [1]:
import pandas as pd
# from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score

import yfinance as yf
from datetime import datetime, timedelta

def get_max_intraday_data(ticker, interval):
    # Set a very large period to attempt to fetch maximum data available
    # Note: Yahoo Finance may limit the actual data returned
    max_period = "60d"
    stock = yf.download(ticker, interval=interval, period='max')
    return stock
data=get_max_intraday_data('^NSEI','1m')
data.to_csv('nifty50_1m_data.csv')

In [2]:
data = pd.read_csv('nifty50_1m_data.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Datetime   1499 non-null   object 
 1   Open       1499 non-null   float64
 2   High       1499 non-null   float64
 3   Low        1499 non-null   float64
 4   Close      1499 non-null   float64
 5   Adj Close  1499 non-null   float64
 6   Volume     1499 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 82.1+ KB
None


In [3]:
print(data.describe())


               Open          High           Low         Close     Adj Close  \
count   1499.000000   1499.000000   1499.000000   1499.000000   1499.000000   
mean   21681.057361  21686.391075  21676.202668  21681.295012  21681.295012   
std       67.231575     66.866434     67.122172     66.804039     66.804039   
min    21433.099609  21468.150391  21433.099609  21452.150391  21452.150391   
25%    21654.500000  21661.174805  21651.650391  21654.699219  21654.699219   
50%    21696.699219  21700.449219  21692.500000  21696.800781  21696.800781   
75%    21724.600586  21730.875000  21719.099609  21724.875000  21724.875000   
max    21829.550781  21832.699219  21808.250000  21830.949219  21830.949219   

       Volume  
count  1499.0  
mean      0.0  
std       0.0  
min       0.0  
25%       0.0  
50%       0.0  
75%       0.0  
max       0.0  


In [4]:
data.isnull().sum()

Datetime     0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [5]:
data.drop(['Volume'],axis=1,inplace=True)
data.head(10)

,Datetime,Open,High,Low,Close,Adj Close
0,2024-01-29 09:15:00+05:30,21433.099609,21477.150391,21433.099609,21476.349609,21476.349609
1,2024-01-29 09:16:00+05:30,21476.449219,21480.349609,21463.449219,21475.800781,21475.800781
2,2024-01-29 09:17:00+05:30,21475.949219,21483.500000,21467.199219,21469.349609,21469.349609
3,2024-01-29 09:18:00+05:30,21467.099609,21479.650391,21462.050781,21474.949219,21474.949219
4,2024-01-29 09:19:00+05:30,21475.300781,21479.400391,21469.699219,21476.300781,21476.300781
5,2024-01-29 09:20:00+05:30,21476.750000,21494.349609,21475.199219,21493.650391,21493.650391
6,2024-01-29 09:21:00+05:30,21494.050781,21508.250000,21494.050781,21495.050781,21495.050781
7,2024-01-29 09:22:00+05:30,21494.349609,21498.849609,21484.699219,21488.000000,21488.000000
8,2024-01-29 09:23:00+05:30,21489.050781,21500.250000,21489.050781,21499.250000,21499.250000
9,2024-01-29 09:24:00+05:30,21500.900391,21511.750000,21498.449219,21506.699219,21506.699219


In [6]:
data.drop(['Datetime','Adj Close'],axis=1,inplace=True)
data

,Open,High,Low,Close
0,21433.099609,21477.150391,21433.099609,21476.349609
1,21476.449219,21480.349609,21463.449219,21475.800781
2,21475.949219,21483.500000,21467.199219,21469.349609
3,21467.099609,21479.650391,21462.050781,21474.949219
4,21475.300781,21479.400391,21469.699219,21476.300781
...,...,...,...,...
1494,21698.650391,21700.000000,21694.800781,21696.199219
1495,21696.150391,21697.449219,21692.349609,21693.949219
1496,21694.699219,21694.699219,21690.650391,21691.400391
1497,21692.000000,21692.000000,21680.099609,21681.150391


In [7]:
X=data.drop(['Close'],axis=1)
print(X)
y=data['Close']


              Open          High           Low
0     21433.099609  21477.150391  21433.099609
1     21476.449219  21480.349609  21463.449219
2     21475.949219  21483.500000  21467.199219
3     21467.099609  21479.650391  21462.050781
4     21475.300781  21479.400391  21469.699219
...            ...           ...           ...
1494  21698.650391  21700.000000  21694.800781
1495  21696.150391  21697.449219  21692.349609
1496  21694.699219  21694.699219  21690.650391
1497  21692.000000  21692.000000  21680.099609
1498  21682.400391  21693.699219  21681.750000

[1499 rows x 3 columns]


In [8]:
model = RandomForestRegressor(n_estimators=100,random_state=0)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [9]:
model.fit(X_train,y_train)


RandomForestRegressor(random_state=0)

In [10]:
y_pred=model.predict(X_test)


In [11]:
model.score(X_test,y_test)

0.9956337970517509

In [12]:
mae=mean_absolute_error(y_test,y_pred)
r2ERROR= r2_score(y_test,y_pred)
print(mae,r2ERROR)

3.0128587239582947 0.9956337970517509


In [14]:
# while True:
import yfinance as yf 
import datetime as dt

ticker = yf.Ticker('^NSEI')
last_pred=0
# while True:
   
data = ticker.history(interval='1m',start='2024-02-01',end='2024-02-02')

data.reset_index(inplace=True)

data.drop(['Datetime','Dividends','Stock Splits','Volume'],axis=1,inplace=True)

y=data['Close']
data.drop(['Close'],axis=1,inplace=True)

prediction=model.predict(data)
last_pred=prediction[-1]
print(f'Prediction ----------> :{ last_pred}')
print(f'mae : {mean_absolute_error(y,prediction)}')
print(f'r2 : {r2_score(y,prediction)}')


Prediction ----------> :21692.0020703125
mae : 1.7075343749999592
r2 : 0.9924688483596561


In [ ]:

import pickle 

pickle.dump(model,open('model_1m.pkl','wb'))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Create a MinMaxScaler object
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the data
scaled_data = scaler.fit_transform(data)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data = scaled_data[0:train_size, :]
test_data = scaled_data[train_size:len(scaled_data), :]

# Split the data into X and y
X_train, y_train = train_data[:, 1:], train_data[:, 0]
X_test, y_test = test_data[:, 1:], test_data[:, 0]

# Reshape the data for LSTM input
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Create the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fit the model to the training data
model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=2)

# Evaluate the model on the testing data
model.evaluate(X_test, y_test)


ModuleNotFoundError: No module named 'tensorflow'